<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/vector_stores/PineconeIndexDemo-0.6.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LlamaIndex + Pinecone

In this tutorial, we show how to use LlamaIndex with Pinecone to answer complex queries over multiple data sources.  
* While Pinecone provides a powerful and efficient retrieval engine,
it remains challenging to answer complex questions that require multi-step reasoning and synthesis over many data sources.
* With LlamaIndex, we combine the power of vector similiarty search and multi-step reasoning to delivery higher quality and richer responses.


Here, we show 2 specific use-cases:
1. compare and contrast queries over Wikipedia articles about different cities.
2. temporal queries that require reasoning over time

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index==0.9.31 pinecone-client==3.0.0.dev10

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Creating a Pinecone Index

In [ ]:
from pinecone import Pinecone, ServerlessSpec

In [ ]:
import os

os.environ[
    "PINECONE_API_KEY"
] = "<Your Pinecone API key, from app.pinecone.io>"


api_key = os.environ["PINECONE_API_KEY"]

pc = Pinecone(api_key=api_key)

In [ ]:
# dimensions are for text-embedding-ada-002

pc.create_index(
    name="quickstart-index",
    dimension=1536,
    metric="euclidean",
    spec=ServerlessSpec(cloud="aws", region="us-west-2"),
)

# If you need to create a PodBased Pinecone index, you could alternatively do this:
#
# from pinecone import Pinecone, PodSpec
#
# pc = Pinecone(api_key='xxx')
#
# pc.create_index(
# 	 name='my-index',
# 	 dimension=1536,
# 	 metric='cosine',
# 	 spec=PodSpec(
# 		 environment='us-east1-gcp',
# 		 pod_type='p1.x1',
# 		 pods=1
# 	 )
# )
#

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '67416a6357dbb65db50c8503e9da4244', 'Date': 'Tue, 16 Jan 2024 05:14:06 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [ ]:
pinecone_index = pc.Index("quickstart-index")

# Use-Case 1: Compare and Contrast

#### Load Dataset

Fetch and load Wikipedia pages

In [ ]:
from llama_index import SimpleDirectoryReader

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [ ]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
]

In [ ]:
from pathlib import Path
import requests

data_path = Path("data_wiki")

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [ ]:
# Load all wiki documents
city_docs = {}
all_docs = []
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[data_path / f"{wiki_title}.txt"]
    ).load_data()
    all_docs.extend(city_docs[wiki_title])

#### Build Indices

In [ ]:
from llama_index import VectorStoreIndex, StorageContext
from llama_index.vector_stores import PineconeVectorStore

In [ ]:
os.environ[
    "OPENAI_API_KEY"
] = "<Your OpenAI API key>"  # Need for storage_context creation

In [ ]:
# Build index for each city document

if "OPENAI_API_KEY" not in os.environ:
    raise EnvironmentError(f"Environment variable OPENAI_API_KEY is not set")

city_indices = {}
index_summaries = {}
for wiki_title in wiki_titles:
    # create storage context
    vector_store = PineconeVectorStore(
        pinecone_index=pinecone_index, namespace=wiki_title
    )
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # build index
    city_indices[wiki_title] = VectorStoreIndex.from_documents(
        city_docs[wiki_title], storage_context=storage_context
    )

    # set summary text for city
    index_summaries[wiki_title] = f"Wikipedia articles about {wiki_title}"

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/21 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/18 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/23 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/21 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors:   0%|          | 0/23 [00:00<?, ?it/s]

#### Build Graph Query Engine for Compare & Contrast Query

In [ ]:
from llama_index.indices.composability import ComposableGraph
from llama_index.indices.keyword_table.simple_base import (
    SimpleKeywordTableIndex,
)

In [ ]:
graph = ComposableGraph.from_indices(
    SimpleKeywordTableIndex,
    [index for _, index in city_indices.items()],
    [summary for _, summary in index_summaries.items()],
    max_keywords_per_chunk=50,
)

In [ ]:
from llama_index.indices.query.query_transform.base import (
    DecomposeQueryTransform,
)
from llama_index.query_engine.transform_query_engine import (
    TransformQueryEngine,
)

decompose_transform = DecomposeQueryTransform(verbose=True)

custom_query_engines = {}
for wiki_title in wiki_titles:
    index = city_indices[wiki_title]
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_metadata={"index_summary": index_summaries[wiki_title]},
    )
    custom_query_engines[index.index_id] = query_engine

custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    retriever_mode="simple",
    response_mode="tree_summarize",
)

In [ ]:
# with query decomposition in subindices
query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

#### Run Compare & Contrast Query

In [ ]:
response = query_engine.query(
    "Compare and contrast the demographics in Seattle, Houston, and Toronto."
)

INFO:llama_index.indices.keyword_table.retrievers:> Starting query: Compare and contrast the demographics in Seattle, Houston, and Toronto.
> Starting query: Compare and contrast the demographics in Seattle, Houston, and Toronto.
INFO:llama_index.indices.keyword_table.retrievers:query keywords: ['seattle', 'compare', 'houston', 'demographics', 'toronto', 'contrast']
query keywords: ['seattle', 'compare', 'houston', 'demographics', 'toronto', 'contrast']
INFO:llama_index.indices.keyword_table.retrievers:> Extracted keywords: ['seattle', 'houston', 'toronto']
> Extracted keywords: ['seattle', 'houston', 'toronto']
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Current query: Compare and contrast the demographics in Seattle, Houston, and Toronto.
> New query: What are the key demographic characteristics of Seattle?
INFO:httpx:HTTP Request: POST https://api.openai.c

In [ ]:
from llama_index.response.pprint_utils import pprint_response

pprint_response(response)

Final Response: Seattle, Houston, and Toronto have diverse
demographics with a mix of ethnic and racial backgrounds. In Seattle,
the population is historically white, but the percentage of white non-
Hispanic residents has been declining. The city also has significant
Asian, Black or African American, Hispanic or Latino, Native American,
Pacific Islander, and mixed-race populations. Houston has a similar
level of diversity, with non-Hispanic whites making up a smaller
percentage of the population compared to Seattle. Houston has a larger
Hispanic or Latino population, along with significant Black or African
American, Asian American, and mixed-race populations. Toronto, on the
other hand, has a different demographic makeup. It has a significant
white population, but also a large number of visible minority groups,
including South Asian, Chinese, Black, Filipino, Arab, and Latin
American populations. Overall, all three cities have diverse
populations, but the specific ethnic and racial gr

# Use-Case 2: Temporal Query

Temporal queries such as "what happened after X" is intuitive to humans, but can often confuse vector databases.  

This is because the vector embedding will focus on the subject "X" rather than the imporant temporal cue. This results in irrelevant and misleading context that harms the final answer.  

LlamaIndex solves this by explicitly maintainging node relationships and leverage LLM to automatically perform query expansion to find more relevant context.  

In [ ]:
!mkdir -p '/content/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/paul_graham/paul_graham_essay.txt' -O '/content/paul_graham/paul_graham_essay.txt'

mkdir: /content: Read-only file system
/content/paul_graham/paul_graham_essay.txt: No such file or directory


In [ ]:
from llama_index import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.vector_stores import PineconeVectorStore


# load documents
documents = SimpleDirectoryReader("/content/paul_graham/").load_data()

# define storage context
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="pg_essay_0.6.0"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# build index
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    # override to store Node in document store in addition to vector store, necessary for the node postprocessor
    store_nodes_override=True,
)

ValueError: Directory /content/paul_graham/ does not exist.

We can define an auto prev/next node postprocessor to leverage LLM reasoning to help query expansion (with relevant additional nodes)

In [ ]:
from llama_index.postprocessor.node import (
    AutoPrevNextNodePostprocessor,
)

# define postprocessor
node_postprocessor = AutoPrevNextNodePostprocessor(
    docstore=index.storage_context.docstore,
    service_context=index.service_context,
    num_nodes=3,
    verbose=True,
)

# define query engine
query_engine = index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[node_postprocessor],
)

#### Example 1

In [ ]:
# Infer that we need to search nodes after current one
response = query_engine.query(
    "What did the author do after handing off Y Combinator to Sam Altman?",
)

In [ ]:
from llama_index.response.pprint_utils import pprint_response

pprint_response(response)

In comparison, naive top-k retrieval results in irrelevant context and hallucinated answer

In [ ]:
# define query engine
naive_query_engine = index.as_query_engine(
    similarity_top_k=1,
)

response = naive_query_engine.query(
    "What did the author do after handing off Y Combinator to Sam Altman?",
)

In [ ]:
pprint_response(response, show_source=True)

#### Example 2

In [ ]:
# Infer that we need to search nodes before current one
response = query_engine.query(
    "What did the author do before handing off Y Combinator to Sam Altman?",
)

In [ ]:
pprint_response(response, show_source=True)